In [ ]:
from financepy.models.finite_difference import option_payoff, calculate_fd_matrix, fd_roll_backwards
from financepy.utils import *
from financepy.market.curves import *
from financepy.products.equity import *
from financepy.models.black_scholes import *

from copy import deepcopy

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
option_type = OptionTypes.AMERICAN_CALL
valuation_date = Date(1, 1, 2015)
expiry_date = valuation_date.add_years(0.5)
time_to_expiry = (expiry_date - valuation_date) / 365.0
risk_free_rate = 0.05
spot_price = 50.0
smooth = digital = False
volatility = 0.20
dividend_yield = 0.05
strike_price = 50.0

s_max = strike_price * 4
dx = 0.2
num_samples = s_max / dx

theta = 0.5

In [ ]:
mu = risk_free_rate - dividend_yield

In [ ]:
# time steps
num_steps = int(num_samples // 2)
dt = time_to_expiry / max(1, num_steps)

TODO: Add dividends (i.e. use drift parameter)

Jacobi

In [ ]:
def calculate_fd_matrix(s, theta, dt, risk_free_rate, volatility):
    j = np.arange(len(s))
    theta_ = 1 - theta
    alpha = 0.5 * dt * theta_ * (volatility**2 * j**2 - mu * j)
    beta = 1 - dt * theta_ * (volatility**2 * j**2 + risk_free_rate)
    kappa = 0.5 * dt * theta_ * (volatility**2 * j**2 + mu * j)
    return np.array([alpha, beta, kappa])


In [ ]:
s = np.arange(s_max, step=dx)
payoff = option_payoff(s, strike_price, smooth, digital, option_type)

In [ ]:
res_ = deepcopy(payoff)[0]
y = [res_]

if option_type in {OptionTypes.EUROPEAN_CALL, OptionTypes.AMERICAN_CALL}:
    f0_old = 0
    fM_old = s_max - strike_price
elif option_type in {OptionTypes.EUROPEAN_PUT, OptionTypes.AMERICAN_PUT}:
    f0_old = strike_price
    fM_old = 0

A = calculate_fd_matrix(s, 1-theta, -dt, risk_free_rate, volatility).T
m = calculate_fd_matrix(s, theta, dt, risk_free_rate, volatility).T

a, b, c = A.T
alpha, beta, kappa = m.T

for i in range(num_steps, -1, -1):
    res_ = deepcopy(y[-1])
    res_ = band_matrix_multiplication(m, 1, 1, res_)
    
    if option_type in {OptionTypes.EUROPEAN_CALL, OptionTypes.AMERICAN_CALL}:
        fM_new = s_max - strike_price * np.exp(-risk_free_rate * (time_to_expiry - i * dt))
        f0_new = f0_old
    elif option_type in {OptionTypes.EUROPEAN_PUT, OptionTypes.AMERICAN_PUT}:
        f0_new = strike_price * np.exp(-risk_free_rate * (time_to_expiry - i * dt)) - s_max
        fM_new = fM_old
    
    # Boundary conditions
    res_[0] += alpha[0] * f0_new - a[0] * f0_old
    res_[-1] += kappa[-1] * fM_new - c[-1] * fM_old
    res = solve_tridiagonal_matrix(A, res_)

    # Early exit for American options
    if option_type in {OptionTypes.AMERICAN_CALL, OptionTypes.AMERICAN_PUT}:
        idx = res < payoff[0]
        res[idx] = payoff[0][idx]
    
    fM_old = fM_new
    y.append(res)
    if not i%10:
        print(f"{i} / {num_steps}")

In [ ]:
import matplotlib.animation as animation
from IPython import display

%matplotlib notebook


In [ ]:
fig, ax = plt.subplots()
line, = ax.plot([], [])
#ax.set_xlim([0, strike_price * 2])
#ax.set_ylim([0, strike_price * 2])

ps = s

def animate(j):
    line.set_data((ps, y[j]))
    return line

animation_1 = animation.FuncAnimation(fig, animate, frames=len(y), interval=1)

plt.plot(ps, y[0])
plt.plot(ps, y[-1])
plt.show()

In [ ]:
sample = np.argmin(np.abs(s-spot_price))
y[-1][sample]